### MCMC
**(1)**

Using the multiple years of data estimate the transition probability matrix for the first-order Markov chain model, with $p_{12}$, the probability of wet given dry and $p_{21}$, the probability of dry given wet. Obtain MLEs and 95\% asymptotic confidence intervals for $p_{12}$ and $p_{21}$.

In [20]:
import numpy as np
import scipy as sp
import scipy.stats
import pandas as pd
import seaborn as sns

my_file = open("snoqualmie_falls.txt", "r")
content = my_file.read().splitlines()
obs = []
for temp in content:
    temp_list = temp.split()
    obs += temp_list
    
## get observations in Jan (be aware of Leap years)
month_years = [31,28,31,30,31,30,31,31,30,31,30,31]
month_leap_years = [31,29,31,30,31,30,31,31,30,31,30,31]
Q1_days = [0,31+28+31]
Q1_leap_days  = [0,31+29+31]

## observations in the first quarter
Q1_obs = []
days = 0
for i in range(1948,1984):
    if (i % 4 == 0):
        temp = obs[(days+Q1_leap_days[0]):(days+Q1_leap_days[1])]
        temp = list(map(int, temp))
        Q1_obs.append(temp)
        days += 366
    else:
        temp = obs[(days+Q1_days[0]):(days+Q1_days[1])]
        temp = list(map(int, temp))
        Q1_obs.append(temp)
        days += 365 
        
## count transition types 
n11 = n10 = n01 = n00 = 0
for i in range(36):
    temp = Q1_obs[i]
    for j in range(len(temp)-1):
        if (temp[j] == 0 and temp[j+1] == 0):
            n00 += 1
        elif (temp[j] == 0 and temp[j+1] > 0):
            n01 += 1
        elif (temp[j] > 0 and temp[j+1] == 0):
            n10 += 1
        else:
            n11 += 1
print("transitions are: \n0 to 0: " + str(n00) + " 0 to 1: " + str(n01) + " \n1 to 0: " + str(n10) + " 1 to 1: " + str(n11))

p01 = n01/(n00+n01)
p10 = n10/(n11+n10)
p00 = n00/(n00 + n01)
p11 = n11/(n11 + n10)

print("MLE are: \n0 to 0: " + str(np.round(p00,4)) + " 0 to 1: " + str(np.round(p01,4)) + " \n1 to 0: " + str(np.round(p10,4)) + " 1 to 1: " + str(np.round(p11,4)))

transitions are: 
0 to 0: 626 0 to 1: 424 
1 to 0: 425 1 to 1: 1738
MLE are: 
0 to 0: 0.5962 0 to 1: 0.4038 
1 to 0: 0.1965 1 to 1: 0.8035


Thus the point estimates are $\hat{p}_{12, MLE} \approx 0.4038$, $\hat{p}_{21, MLE} \approx 0.1965$.


To calculate the asymptotic $95\%$ confidence interval for $\hat{p}_{12, MLE}$ and $\hat{p}_{21, MLE}$, use the asymptotic normality property of the MLE estimate(this should hold since the markov chain is of finite states and irreducible):

$$
\frac{\hat{p}_{ij} - p_{ij}}{\sqrt{p_{ij}(1-p_{ij})}}\sqrt{n\pi_i} \xrightarrow{\text{D}}N(0,1)
$$

In [5]:
## solving the stationary distribution
tpm = np.array([[p00, p01],[p10, p11]])
A_temp = np.transpose(tpm) - np.identity(2)
A = np.vstack([A_temp[1],np.ones(2)])
#A = np.array(tpm[1], np.ones(2))
b = np.array([0,1])
pi_hat = np.linalg.solve(A,b)
n_total = n00 + n10 + n01 + n11
std_01 = np.sqrt(tpm[0,1]*(1-tpm[0,1]))/np.sqrt(n_total)/np.sqrt(pi_hat[0])
std_10 = np.sqrt(tpm[1,0]*(1-tpm[1,0]))/np.sqrt(n_total)/np.sqrt(pi_hat[1])
# CI for p12
print("95% confidence interval for p12: (", str(np.round(p01 - 1.96*std_01,4)), ",", str(np.round(p01 + 1.96*std_01,4)), ")")
# CI for p21
print("95% confidence interval for p21: (", str(np.round(p10 - 1.96*std_10,4)), ",", str(np.round(p10 + 1.96*std_10,4)), ")")

95% confidence interval for p12: ( 0.3742 , 0.4335 )
95% confidence interval for p21: ( 0.1797 , 0.2132 )


$$
p_{i j} \in\left[\hat{p}_{i j}-z_{1-\alpha / 2} \frac{\sqrt{\hat{p}_{i j}\left(1-\hat{p}_{i j}\right)}}{\sqrt{n \hat{\pi}_{i}}}, \hat{p}_{i j}+z_{1-\alpha / 2} \frac{\sqrt{\hat{p}_{i j}\left(1-\hat{p}_{i j}\right)}}{\sqrt{n \hat{\pi}_{i}}}\right]
$$

Thus the confidence interval for $p_{12}, p_{21}$ are $( 0.3742 , 0.4335 )$ and $( 0.1797 , 0.2132 )$.

**(b)**

Under the assumption of independent uniform priors, give the
	form of the posterior distributions for the parameters
	$p_{12},p_{21}$. Obtain the posterior and report posterior medians
	and 95\% credible intervals for each of the two parameters. (use any package to compute the inverse of the cumulative distribution of the posterior to find the median and the credible interval).


**Solution:**
Notice that for each row in tpm, we have

$$
\begin{aligned}
p_{i 1}, p_{i 2} & \sim \operatorname{Dirichlet}(1,1), \quad i=1,2 \\
\left(p_{11}, p_{12}\right) &\perp\left(p_{21}, p_{22}\right) 
\end{aligned}
$$

Since 

$$
\begin{aligned}
\pi\left(p_{i} \mid X_{1}, \ldots, X_{n}\right) & \propto p_{i 1}^{n_{i 1}+1-1} p_{i 2}^{n_{i 2}+1-1}, \quad i=1,2 \\
p_{i} \mid X_{1}, \ldots, X_{n} & \sim \operatorname{Dirichlet}\left(n_{i 1}+1, n_{i 2}+1\right)
\end{aligned}
$$

As we have $n_{12} = 424, n_{21} = 425$, the posterior distributions are 

$$
p_{12}\left|X_{1}, \ldots, X_{n} \sim \operatorname{Beta}(n_{12} + 1, n_{11} + 1),\left(p_{21}\right)\right| X_{1}, \ldots, X_{n} \sim \operatorname{Beta}(n_{21}+1, n_{22} + 1)
$$

In [12]:
rv_12 = sp.stats.beta(n01 + 1, n00 + 1)
rv_21 = sp.stats.beta(n10 + 1, n11 + 1)
print("95% posterior credible interval for p_12: ( " + str(np.round(rv_12.ppf(0.025),3)) +
     "," + str(np.round(rv_12.ppf(0.975),3)), ")")
print("95% posterior credible interval for p_21: ( " + str(np.round(rv_21.ppf(0.025),3)) +
     "," + str(np.round(rv_21.ppf(0.975),3)), ")")
print("posterior median for p_12: " + str(np.round(rv_12.ppf(0.5),4)))
print("posterior median for p_21: " + str(np.round(rv_21.ppf(0.5),4)))

95% posterior credible interval for p_12: ( 0.375,0.434 )
95% posterior credible interval for p_21: ( 0.18,0.214 )
posterior median for p_12: 0.4039
posterior median for p_21: 0.1967


**(c)**
Read carefully Sec. 11.5 of Lecture 11, test the independence of the data in a frequentist and a Bayesian viewpoint (for the Bayesian viewpoint you can use the prior of your choice).

Conduct the hypothesis test of independent samples against Markov chain samples. Notice that the formula of likelihood ratio is given by

$$
T_{n}=2\left(l_{n}\left(\hat{\boldsymbol{P}}_{M L E}\right)-l_{n}\left(\hat{\boldsymbol{P}}_{M L E, 0}\right)\right)=2 \sum_{i=1}^{2} \sum_{j=1}^{2} n_{i j} \log \left(\frac{n_{i j} \cdot n}{n_{i+} \cdot n_{+j}}\right)=501.945
$$

In [13]:
T = n00*(np.log(n00) + np.log(n_total) - np.log(n01 + n00) - np.log(n00 + n10)) + n01*(np.log(n01) + np.log(n_total) - np.log(n01 + n00) - np.log(n01 + n11))+ n10*(np.log(n10) + np.log(n_total) - np.log(n10 + n11) - np.log(n00 + n10))+ n11*(np.log(n11) + np.log(n_total) - np.log(n10 + n11) - np.log(n01 + n11))
T *= 2
T

501.9457978794331

In [14]:
rv_chisq = sp.stats.chi(1)
1-rv_chisq.cdf(T)

0.0

Thus we reject the null hypothesis that the data are IID samples with at least 95 % confidence.
To conduct this test under Bayesian prospective, pose a uniform distribution on the each row of the transition probability matrix and assume independence between two rows. The Bayes factor is given by

$$
B F=\frac{B\left(n_{+1}+1, n_{+2}+1\right) / B(1,1)}{\prod_{i=1}^{2}\left[B\left(n_{i 1}+1, n_{i 2}+1\right) / B(1,1)\right]} \approx 2.577 \times 10^{-108}
$$

The Bayes factor is extremely close to zero, suggesting that the data is not independent. 